<a href="https://colab.research.google.com/github/sujay-dsa/ComputerVision/blob/master/EVA_5_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [74]:
# Modified snippet
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 15, 3) #input -28 OUtput - 26 RF -3
        self.conv2 = nn.Conv2d(15, 20, 3) #input -26 OUtput - 24 RF -5
        self.conv3 = nn.Conv2d(20, 25, 3) #input -24 OUtput - 22 RF -7

        self.batchnn1=nn.BatchNorm2d(25)
        self.pool1 = nn.MaxPool2d(2, 2) #input -22 OUtput - 11 RF -14
        
        self.conv4 = nn.Conv2d(25, 25, 3) #input -11 OUtput - 9 RF -16
        self.conv5 = nn.Conv2d(25, 25, 3) #input -9 OUtput - 7 RF -18
        self.conv6 = nn.Conv2d(25, 20, 3) #input -7 OUtput - 5 RF -20
        self.conv7 = nn.Conv2d(20, 16, 3) #input -5 OUtput - 3 RF -22
        self.conv8 = nn.Conv2d(16, 10, 3) #input -3 OUtput - 1 RF -22

    def forward(self, x):
        x = F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x))))))
        #x = self.pool1(self.batchnn1(x))
        x = self.pool1(F.relu(self.batchnn1(x)))
        x = F.relu(self.conv5(F.relu(self.conv4(x))))
        x = F.relu(self.conv7(F.relu(self.conv6(x))))
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [73]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 15, 26, 26]             150
            Conv2d-2           [-1, 20, 24, 24]           2,720
            Conv2d-3           [-1, 25, 22, 22]           4,525
       BatchNorm2d-4           [-1, 25, 22, 22]              50
         MaxPool2d-5           [-1, 25, 11, 11]               0
            Conv2d-6             [-1, 30, 9, 9]           6,780
            Conv2d-7             [-1, 25, 7, 7]           6,775
            Conv2d-8             [-1, 20, 5, 5]           4,520
            Conv2d-9             [-1, 16, 3, 3]           2,896
           Conv2d-10             [-1, 10, 1, 1]           1,450
Total params: 29,866
Trainable params: 29,866
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.41
Params size (MB): 0.11
Estimated Tot

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [59]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [63]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [65]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

100%|██████████| 469/469 [00:10<00:00, 43.41it/s]



Test set: Average loss: 0.0862, Accuracy: 9730/10000 (97.300%)

